### Wind suitability based off of potential wind at varying hub heights and a capacity factor of 30 percent

Data for potential wind capacity for 80m, 110m, and 140m hub heights for current technology assumptions downloaded from https://maps.nrel.gov/wind-prospector

Metadata for potential resource capacity files:  https://edacarc20.unm.edu/arcgis/rest/services/nmenergytool/EDAC_Energy/FeatureServer/0



In [2]:
import os
import glob

import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd


In [11]:
# suitabilty data directory
data_dir = '/Users/d3y010/projects/cerf/suitability/data'

# template raster from CERF
template_raster = os.path.join(data_dir, 'reference', 'cerf_landmask.tif')

# exclusion data for wind dir
exclusion_dir = os.path.join(data_dir, 'wind', 'exclusions')

# directory where downloaded data was decompressed
download_dir = os.path.join(data_dir, 'wind', 'nrel')

# processed shapefile directory
wind_shp_dir = os.path.join(data_dir, 'wind', 'shp')

# directory to write potential capacity rasters to
potential_dir = os.path.join(data_dir, 'wind', 'potential_suitability')

# where to write the final suitability outputs to
output_dir = os.path.join(data_dir, 'wind')


In [7]:
# get cerf CRS from template raster
with rasterio.open(template_raster) as src:
    target_crs = src.crs


### Extract desired data from downloaded shapefiles and save to new files

In [8]:
%%time

# list of hub heights to process
hub_height_list = ['080', '110', '140']

for hub_height in hub_height_list:

    # input shapefile
    f = os.path.join(download_dir, f'pot_wind_cap_{hub_height}_current', f'pot_wind_cap_{hub_height}_current.shp')

    # only keep necessary fields
    gdf = gpd.read_file(f)[['area_km2', 'a30', 'geometry']].to_crs(target_crs)

    # get polygons with at least 1 km2 of available development area at 30% capacity
    gdx = gdf.loc[gdf['a30'] > 1].copy()

    # assign suitability value
    gdx['value'] = 0
    
    file_name = f'cerf_nrel_wind_development_potential_hubheight{hub_height}m_cf30.shp'
    output_file = os.path.join(wind_shp_dir, file_name)
    
    # write output
    gdx[['value', 'geometry']].to_file(output_file)


CPU times: user 27.8 s, sys: 615 ms, total: 28.4 s
Wall time: 28.7 s


### Rasterize shapefiles for potential

In [10]:
%%time 

shp_list = glob.glob(os.path.join(wind_shp_dir, 'cerf_nrel_wind_development_potential_hubheight*m_cf30.shp'))

for i in shp_list:
        
    basename = os.path.basename(i)
    base_noext = os.path.splitext(basename)[0]
    
    output_raster = os.path.join(potential_dir, f"{base_noext}.tif")

    gdal_rasterize_cmd = f"gdal_rasterize -l {base_noext} -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff {i} {output_raster}"

    print(gdal_rasterize_cmd, '\n')
    
    os.system(gdal_rasterize_cmd)


gdal_rasterize -l cerf_nrel_wind_development_potential_hubheight140m_cf30 -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff /Users/d3y010/projects/cerf/suitability/data/wind/shp/cerf_nrel_wind_development_potential_hubheight140m_cf30.shp /Users/d3y010/projects/cerf/suitability/data/wind/potential_suitability/cerf_nrel_wind_development_potential_hubheight140m_cf30.tif 

gdal_rasterize -l cerf_nrel_wind_development_potential_hubheight080m_cf30 -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff /Users/d3y010/projects/cerf/suitability/data/wind/shp/cerf_nrel_wind_development_potential_hubheight080m_cf30.shp /Users/d3y010/projects/cerf/suitability/data/wind/potential_suitability/cerf_nrel_wind_development_potential_hubheight080m_cf30.tif 

gdal_rasterize -l cerf_nrel_wind_development_potential_hubheight110m_cf30 -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.83

### Apply exclusions

In [13]:
%%time 

# non-potential suitability data that applies to wind
exclusion_files = [os.path.join(exclusion_dir, i) for i in os.listdir(exclusion_dir) if os.path.splitext(i)[-1] in ('.tif', '.img')]

# combine exclusion layers into a single array
for idx, i in enumerate(exclusion_files):
    
    with rasterio.open(i) as src:
        
        if idx == 0:
            arr = src.read(1)
            
        else:
            arr += src.read(1)


CPU times: user 347 ms, sys: 49.9 ms, total: 397 ms
Wall time: 766 ms


In [14]:
# apply to each potential layer
potential_rasters = glob.glob(os.path.join(potential_dir, 'cerf_nrel_wind_development_potential_hubheight*m_cf30.tif'))

for i in potential_rasters:

    with rasterio.open(i) as src:

        # update metadata datatype to int16
        metadata = src.meta.copy()
        metadata.update({'dtype': rasterio.int16})
        
        # get array of potential suitability
        arx = src.read(1)
        
        # apply exclusions
        arx += arr
        
        # reclassify to 0 (suitable), 1 (unsuitable)
        arx = np.where(arx == 0, 0, 1)
        
        # write aggregate tech-specific suitability layer to file
        hub_height_str = i.split('_')[-2]
        final_raster = os.path.join(output_dir, f'cerf_wind_onshore_{hub_height_str}.tif')
        
        with rasterio.open(final_raster, 'w', **metadata) as dest:
            dest.write(arx.astype(rasterio.int16), 1)
